In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./source/etl'))

from pyspark.sql import SparkSession
from SparkDBUtils import SparkDB
import delta
import datetime as dt
import pyspark.sql
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StructType, StructField, IntegerType, TimestampType

sparkdb = SparkDB()
spark = sparkdb.spark

In [2]:
spark.sql("show tables in default").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|            date_dim|      false|
|  default|precio_dia_agg_no...|      false|
|  default|   producto_dia_fact|      false|
|  default|        producto_dim|      false|
|  default|       sequences_cfg|      false|
+---------+--------------------+-----------+



In [6]:
spark.sql("select * from sequences_cfg").show()

+------------+----+--------------------+
|  table_name|  id|             ts_load|
+------------+----+--------------------+
|producto_dim|6871|2023-02-26 19:32:...|
|    date_dim|  39|2023-02-26 19:32:...|
+------------+----+--------------------+



In [7]:
spark.sql("select * from producto_dia_fact").count()

224904

In [8]:
spark.sql("select * from precio_dia_agg_norm_fact").count()

39

In [14]:
producto_dia_fact = spark.table("producto_dia_fact").alias("pf")
date_dim = spark.table("date_dim").alias("d")
producto_dim = spark.table("producto_dim").alias("p")

In [18]:
p = producto_dia_fact\
    .join(date_dim, "id_date")\
    .join(producto_dim, "id_producto")

p.select(f.column("pf.price"),
        f.column("d.date"),
        f.column("p.product")).show(10)

+------------------+----------+--------------------+
|             price|      date|             product|
+------------------+----------+--------------------+
| 1.590000033378601|2022-11-21|VIÑA DANZA vino r...|
| 1.649999976158142|2022-11-21|DIA SNACK MANIAC ...|
| 3.690000057220459|2022-11-21|DANONE ACTIVIA bí...|
|0.9900000095367432|2022-11-21|LA CONFIANZA hiel...|
|2.1500000953674316|2022-11-21|SMINT pastillas d...|
| 2.190000057220459|2022-11-21|SELECCIÓN DE DIA ...|
| 5.789999961853027|2022-11-21|CILLIT BANG limpi...|
|1.4900000095367432|2022-11-21|Jengibre bandeja ...|
|             18.75|2022-11-21|FAUSTINO vino tin...|
| 1.149999976158142|2022-11-21|EL MOLINO DE DIA ...|
+------------------+----------+--------------------+
only showing top 10 rows



224904